In [33]:
import numpy as np
import scipy.stats as ss

mu = 721
n = 20
empleados = [710,732,654,760,774,675,757,718,814,755,736,774,768,770,677,747,823,660,797,796]

media = round(np.array(empleados).mean())
des = round(np.array(empleados).std())

#a
a = 0.1

#Ho : mu = 721
#Hi : mu != 721

t = (media - mu) / (des/20**0.5)

tCritico = ss.t.ppf(1-a, 19)

print(f"Estadistico t : {t}")
print(f"Estadistivo t critico {tCritico}\n")

print(f"Como el valor t es mayor que el tCritico (t > tCritico), se puede concluir que la hipotesis nula es rechazada porque el promedio de consumo del estudio es superior a la media de consumo de la provincia\n")

#b
p = round(ss.t.sf(t, 19)*2,3)
p

print(f"El valor p obtenido es de {p}, el cual es menor a 0.5")

Estadistico t : 2.23606797749979
Estadistivo t critico 1.3277282090267986

Como el valor t es mayor que el tCritico (t > tCritico), se puede concluir que la hipotesis nula es rechazada porque el promedio de consumo del estudio es superior a la media de consumo de la provincia

El valor p obtenido es de 0.038, el cual es menor a 0.5
